Kategorizace oblastí dle výše příjmů na osobu. Výběr názvů oblastí, ve kterých pod hranicí chudoby žije více než 20% domácností.

*Rozhraní DB-API, parametrizace, uložení dotazu do proměnné, funkce CASE, CTE, načtení výstupu do formátu DataFrame*

In [5]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('chicago_database.db')
cursor = conn.cursor()

query = """
    WITH income_level_categorised AS
        (SELECT COMMUNITY_AREA_NAME, PER_CAPITA_INCOME, PERCENT_HOUSEHOLDS_BELOW_POVERTY,
            CASE
                WHEN PER_CAPITA_INCOME < 24000 THEN "Very Low Income Area"
                WHEN PER_CAPITA_INCOME BETWEEN 24001 AND 40000 THEN "Low Income Area"
                WHEN PER_CAPITA_INCOME BETWEEN 40001 AND 56000 THEN "Mid Income Area"
                WHEN PER_CAPITA_INCOME BETWEEN 56001 AND 70000 THEN "High Income Area"
                WHEN PER_CAPITA_INCOME > 70001 THEN "Very High Income Area"
                ELSE "Outlier"
            END AS INCOME_LEVEL
        FROM census)

    SELECT * FROM income_level_categorised
    WHERE PERCENT_HOUSEHOLDS_BELOW_POVERTY > ?
    ORDER BY PER_CAPITA_INCOME DESC
    """

percent_households_below_poverty = (20,)

results = pd.read_sql_query(query, conn, params = percent_households_below_poverty)
print(results.head(5))

cursor.close()
conn.close()


  COMMUNITY_AREA_NAME  PER_CAPITA_INCOME  PERCENT_HOUSEHOLDS_BELOW_POVERTY  \
0      Near West Side              44689                              20.6   
1             Kenwood              35911                              21.7   
2              Uptown              35787                              24.0   
3         Rogers Park              23939                              23.6   
4             Douglas              23791                              29.6   

           INCOME_LEVEL  
0       Mid Income Area  
1       Low Income Area  
2       Low Income Area  
3  Very Low Income Area  
4  Very Low Income Area  


Výběr škol z oblastí s vyšším počtem trestných činů, než je průměrný počet trestných činů pro všechny oblasti. Vybrané školy současně získaly certifikaci "Healthy School".

*Rozhraní Jupyter Notebook - SQL Magic, View, Implicit Join, Nested Select*

In [6]:
%load_ext sql
import prettytable
prettytable.DEFAULT = 'DEFAULT'

%sql sqlite:///chicago_database.db

In [7]:
%%sql

CREATE VIEW area_crime_totals AS
SELECT COMMUNITY_AREA_NUMBER, COUNT(DISTINCT id) AS CRIMES_PER_AREA
FROM crime
GROUP BY COMMUNITY_AREA_NUMBER;

 * sqlite:///chicago_database.db
Done.


[]

In [8]:
%%sql

SELECT s.School_ID, s.NAME_OF_SCHOOL, s.COMMUNITY_AREA_NAME, a.CRIMES_PER_AREA
FROM schools s, area_crime_totals a
WHERE s.COMMUNITY_AREA_NUMBER = a.COMMUNITY_AREA_NUMBER
    AND a.CRIMES_PER_AREA > (SELECT AVG(CRIMES_PER_AREA) FROM area_crime_totals)
    AND UPPER(s.HEALTHY_SCHOOL_CERTIFIED) = "YES"
ORDER BY a.CRIMES_PER_AREA DESC
LIMIT 5

 * sqlite:///chicago_database.db
Done.


School_ID,NAME_OF_SCHOOL,COMMUNITY_AREA_NAME,CRIMES_PER_AREA
610320,Ana Roque de Duprey Elementary School,WEST TOWN,9
610313,Jose De Diego Elementary Community Academy,WEST TOWN,9
610125,Irma C Ruiz Elementary School,LOWER WEST SIDE,8
610216,John A Walsh Elementary School,LOWER WEST SIDE,8
609872,Manuel Perez Elementary School,LOWER WEST SIDE,8


Výběr názvů oblastí, číselných označení oblastí a indexu HARDSHIP (skóre shrnující vícero socioekonomických indikátorů). Pokud HARDSHIP index není u oblasti uveden, bude nahrazen průměrnou hodnotou HARDSHIP indexu všech oblastí. 

Výběr zahrnuje pouze ty oblasti, pro které neexistuje záznam přečinu typu “DECEPTIVE PRACTISE”.

*Rozhraní Jupyter Notebook - SQL Magic, NULL-Function, Nested Select, Explicit Join*

In [9]:
%%sql

SELECT
    s.COMMUNITY_AREA_NUMBER, s.COMMUNITY_AREA_NAME,
    IFNULL(s.HARDSHIP_INDEX,
           (SELECT AVG(HARDSHIP_INDEX) FROM census s)) 
       AS HARDSHIP_INDEX_CORRECTED
FROM census s
    LEFT JOIN crime e ON s.COMMUNITY_AREA_NUMBER = e.COMMUNITY_AREA_NUMBER AND e.PRIMARY_TYPE = "DECEPTIVE PRACTICE"
WHERE e.PRIMARY_TYPE IS NULL
LIMIT 5


 * sqlite:///chicago_database.db
Done.


COMMUNITY_AREA_NUMBER,COMMUNITY_AREA_NAME,HARDSHIP_INDEX_CORRECTED
3.0,Uptown,20.0
4.0,Lincoln Square,17.0
5.0,North Center,6.0
7.0,Lincoln Park,2.0
9.0,Edison Park,8.0


Výpočet celkového a kumulativního počtu trestných činů v jednotlivých měsících.

*Rozhraní SQL Magic, SQLite Date Functions, Window Function, CTE*

In [10]:
%%sql

WITH monthly_crimes AS (
       SELECT strftime ('%Y', DATE) AS YEAR,
              strftime ('%m', DATE) AS MONTH,
              COUNT(DISTINCT ID) AS CRIMES_TOTAL
       FROM crime
       GROUP BY YEAR, MONTH)


SELECT 
       YEAR, 
       MONTH, 
       CRIMES_TOTAL, 
       SUM (CRIMES_TOTAL) OVER (ORDER BY YEAR, MONTH) 
               AS CUMULATIVE_CRIMES_TOTAL
FROM monthly_crimes
LIMIT(10)


 * sqlite:///chicago_database.db
Done.


YEAR,MONTH,CRIMES_TOTAL,CUMULATIVE_CRIMES_TOTAL
2001,09,1,1
2002,04,3,4
2002,05,3,7
2002,06,3,10
2002,07,2,12
2002,08,3,15
2002,09,4,19
2002,10,6,25
2002,12,5,30
2003,01,1,31
